In [80]:
import os
import sys
import pprint
import pandas as pd

base_dir = 'F:\Thesis_ssd\MasterThesis3.0'
os.chdir(base_dir)

from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.DataGenerator import DataGenerator
from Classes.Modeling.GridSearchResultProcessor import GridSearchResultProcessor
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
from Classes.Modeling.ResultFitter import ResultFitter
import json

In [16]:
load_args = {
    'earth_explo_only' : True,
    'noise_earth_only' : False,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 0.5,
    'seed' : 1,
    'subsample_size' : 0.25
}

loadData = LoadData(**load_args)

full_ds, train_ds, val_ds, test_ds = loadData.get_datasets()
noise_ds = loadData.noise_ds
handler = DataHandler(loadData)
dataGen = DataGenerator(loadData)
resultFitter = ResultFitter(loadData)

AttributeError: 'LoadData' object has no attribute 'getDatasets'

In [6]:
result_file_name = 'results_6_noiseNotNoise_timeAug_sscale_noiseAug_earlyS.csv'
model = resultFitter.fit_from_csv_and_index(result_file_name, 0, 2, 10, use_tensorboard = False, 
                               use_liveplots = True, use_custom_callback = False)

NameError: name 'resultFitter' is not defined

In [244]:
def f1_score(precision, recall):
    f1 = 2*((precision*recall)/(precision + recall))
    return f1

def create_f1_list(precision_df, recall_df):
    f1 = []
    for i in range(len(precision_df)):
        f1.append(f1_score(precision_df.loc[i], recall_df.loc[i]))
    return f1

result_file_name = 'results_LSTM_NARROW_noiseNotNoise_timeAug_sscale_noiseAug_earlyS.csv'
df = GridSearchResultProcessor().get_results_df_by_name(result_file_name, 2)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

df_f1 = df.copy()
df_f1.columns=df_f1.columns.str.strip()
all_train_precision = df_f1['train_precision']
all_train_recall = df_f1['train_recall']
all_val_precision = df_f1['val_precision']
all_val_recall = df_f1['val_recall']
f1_train = create_f1_list(all_train_precision, all_train_recall)
f1_val = create_f1_list(all_val_precision, all_val_recall)
df_f1['train_f1'] = f1_train
df_f1['val_f1'] = f1_val
df_f1

,batch_size,epochs,learning_rate,num_channels,num_layers,optimizer,activation,decay_sequence,dropout_rate,filters,kernel_size,l1_r,l2_r,output_layer_activation,padding,start_neurons,use_layerwise_dropout_batchnorm,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall,train_f1,val_f1
0,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00050,0.0010,sigmoid,same,32,True,1.181961,0.840200,0.802052,0.864571,1.242921,0.801153,0.742661,0.806089,0.832139,0.773076
1,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,0.676618,0.950296,0.901711,0.898102,0.906589,0.871457,0.787405,0.791229,0.899903,0.789312
2,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00005,0.0010,sigmoid,same,32,True,0.683111,0.955232,0.910079,0.906246,0.899621,0.883255,0.808475,0.814426,0.908158,0.811440
3,32,40,0.100,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,1.038368,0.882141,0.842143,0.878161,1.142973,0.830189,0.777699,0.829286,0.859775,0.802664
4,32,40,0.050,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,0.912541,0.852709,0.842568,0.920599,1.009884,0.801129,0.776278,0.874592,0.879857,0.822508
5,32,40,0.025,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,0.822398,0.961424,0.865592,0.829742,0.975978,0.905702,0.784801,0.748460,0.847288,0.766200
6,32,40,0.005,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,0.757576,0.981014,0.890980,0.851289,1.015217,0.918211,0.785038,0.736499,0.870682,0.759994
7,24,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,0.705723,0.958900,0.888810,0.868868,0.900751,0.895990,0.795691,0.777456,0.878726,0.786468
8,28,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,0.760744,0.962676,0.887654,0.863255,0.932806,0.890008,0.786187,0.767560,0.875284,0.776762
9,34,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,0.642977,0.980930,0.905192,0.873343,0.905566,0.913580,0.791271,0.751814,0.888982,0.771038


In [245]:
sorted_f1_df = df_f1.sort_values(by=['val_f1', 'train_f1'], axis = 0, ascending=False)
sorted_f1_df

,batch_size,epochs,learning_rate,num_channels,num_layers,optimizer,activation,decay_sequence,dropout_rate,filters,kernel_size,l1_r,l2_r,output_layer_activation,padding,start_neurons,use_layerwise_dropout_batchnorm,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall,train_f1,val_f1
4,32,40,0.050,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,0.912541,0.852709,0.842568,0.920599,1.009884,0.801129,0.776278,0.874592,0.879857,0.822508
2,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00005,0.0010,sigmoid,same,32,True,0.683111,0.955232,0.910079,0.906246,0.899621,0.883255,0.808475,0.814426,0.908158,0.811440
16,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,0.744326,0.956039,0.906676,0.900043,0.955661,0.883629,0.807292,0.811888,0.903347,0.809583
31,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.25,17,5,0.00010,0.0010,sigmoid,same,32,True,0.704676,0.949767,0.915374,0.920527,0.985820,0.870893,0.801136,0.816600,0.917943,0.808794
20,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0001,sigmoid,same,32,True,0.640559,0.968565,0.923081,0.913165,0.896235,0.897810,0.811316,0.802465,0.918096,0.806866
10,36,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,0.654485,0.957651,0.916667,0.914200,0.891374,0.880332,0.803181,0.809074,0.915432,0.806117
22,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,24,True,0.579095,0.946154,0.910363,0.916332,0.824668,0.870478,0.797822,0.811163,0.913338,0.804437
3,32,40,0.100,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,1.038368,0.882141,0.842143,0.878161,1.142973,0.830189,0.777699,0.829286,0.859775,0.802664
32,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.20,17,5,0.00010,0.0010,sigmoid,same,32,True,0.738488,0.957885,0.915658,0.912327,1.017721,0.875744,0.794981,0.799565,0.913990,0.797266
25,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,34,True,0.676466,0.963020,0.910363,0.898754,0.919012,0.895473,0.801847,0.788692,0.904521,0.795215


In [240]:
sorted_f1_df = df_f1.sort_values(by=['val_precision', 'train_precision'], axis = 0, ascending=False)
sorted_f1_df

,batch_size,epochs,learning_rate,num_channels,num_layers,optimizer,activation,decay_sequence,dropout_rate,filters,kernel_size,l1_r,l2_r,output_layer_activation,padding,start_neurons,use_layerwise_dropout_batchnorm,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall,train_f1,val_f1
20,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0001,sigmoid,same,32,True,0.640559,0.968565,0.923081,0.913165,0.896235,0.897810,0.811316,0.802465,0.918096,0.806866
2,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00005,0.0010,sigmoid,same,32,True,0.683111,0.955232,0.910079,0.906246,0.899621,0.883255,0.808475,0.814426,0.908158,0.811440
16,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,0.744326,0.956039,0.906676,0.900043,0.955661,0.883629,0.807292,0.811888,0.903347,0.809583
24,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,30,True,0.670242,0.974445,0.903272,0.876477,0.877753,0.911478,0.806818,0.779993,0.889673,0.793179
11,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,0.639839,0.975650,0.908803,0.883899,0.882494,0.909860,0.805398,0.779268,0.896178,0.792117
10,36,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,0.654485,0.957651,0.916667,0.914200,0.891374,0.880332,0.803181,0.809074,0.915432,0.806117
25,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,34,True,0.676466,0.963020,0.910363,0.898754,0.919012,0.895473,0.801847,0.788692,0.904521,0.795215
23,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,28,True,0.616975,0.971631,0.907148,0.885229,0.858172,0.899334,0.801373,0.783617,0.896055,0.792396
31,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.25,17,5,0.00010,0.0010,sigmoid,same,32,True,0.704676,0.949767,0.915374,0.920527,0.985820,0.870893,0.801136,0.816600,0.917943,0.808794
18,32,40,0.010,3,2,sgd,tanh,"[1, 2]",0.30,17,5,0.00010,0.0010,sigmoid,same,32,True,0.673688,0.965807,0.901239,0.881658,0.875211,0.902982,0.801136,0.779268,0.891341,0.790051


In [236]:
df[df.columns[:15]].iloc[40:]

,batch_size,epochs,learning_rate,num_channels,num_layers,optimizer,activation,decay_sequence,dropout_rate,filters,kernel_size,l1_r,l2_r,output_layer_activation,padding


In [160]:
df[df.columns[15:]].iloc[40:]

,train_loss,train_precision,train_accuracy,train_recall,val_loss,val_precision,val_accuracy,val_recall
40,1.084770,0.782372,0.786742,0.937814,1.147155,0.755786,0.754972,0.923161
41,0.899954,0.950041,0.860890,0.833142,1.037570,0.891613,0.777936,0.751359
42,1.925948,0.747005,0.749432,0.938575,1.981802,0.707387,0.688447,0.891990
43,2.269218,0.669505,0.519271,0.535989,2.274575,0.654939,0.509470,0.526278
44,2.271817,0.659082,0.502983,0.511301,2.273199,0.650253,0.501420,0.512142
45,0.806053,0.964770,0.904025,0.887121,1.025919,0.893439,0.792614,0.774918
46,0.841913,0.955342,0.908333,0.903420,1.085873,0.875353,0.787405,0.786517
47,0.910195,0.951840,0.911742,0.912532,1.132424,0.878223,0.798295,0.802465
48,0.988093,0.953485,0.908523,0.905616,1.212564,0.877102,0.792850,0.794128
49,1.054804,0.920381,0.908570,0.943146,1.275719,0.836668,0.785038,0.833635


## Findings:
 - New optimal dropout_rate: 0.2 (train_acc: 98%, val_acc: 90%) Old 0.3 (train_acc: 96.8%, val_acc: 90.9%)
 - RELU and Sigmoid activation functions do not work with this model.
 - L1 regularization is counter productive (Less is more) so let l1 = 0.00
 - 32 start neurons appear to be optimal

In [161]:
df.iloc[40:]

,batch_size,epochs,learning_rate,num_layers,optimizer,activation,decay_sequence,dropout_rate,filters,kernel_size,l1_r,l2_r,output_layer_activation,padding,start_neurons,train_loss,train_precision,train_accuracy,train_recall,val_loss,val_precision,val_accuracy,val_recall
40,64,33,0.20000,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,32,1.084770,0.782372,0.786742,0.937814,1.147155,0.755786,0.754972,0.923161
41,64,33,0.10000,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,32,0.899954,0.950041,0.860890,0.833142,1.037570,0.891613,0.777936,0.751359
42,64,33,0.00100,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,32,1.925948,0.747005,0.749432,0.938575,1.981802,0.707387,0.688447,0.891990
43,64,33,0.00010,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,32,2.269218,0.669505,0.519271,0.535989,2.274575,0.654939,0.509470,0.526278
44,64,33,0.00001,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,32,2.271817,0.659082,0.502983,0.511301,2.273199,0.650253,0.501420,0.512142
45,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,30,0.806053,0.964770,0.904025,0.887121,1.025919,0.893439,0.792614,0.774918
46,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,32,0.841913,0.955342,0.908333,0.903420,1.085873,0.875353,0.787405,0.786517
47,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,36,0.910195,0.951840,0.911742,0.912532,1.132424,0.878223,0.798295,0.802465
48,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,40,0.988093,0.953485,0.908523,0.905616,1.212564,0.877102,0.792850,0.794128
49,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,44,1.054804,0.920381,0.908570,0.943146,1.275719,0.836668,0.785038,0.833635


In [162]:
def f1_score(precision, recall):
    f1 = 2*((precision*recall)/(precision + recall))
    return f1

def create_f1_list(precision_df, recall_df):
    f1 = []
    for i in range(len(precision_df)):
        f1.append(f1_score(precision_df.loc[i], recall_df.loc[i]))
    return f1

,batch_size,epochs,learning_rate,num_layers,optimizer,activation,decay_sequence,dropout_rate,filters,kernel_size,l1_r,l2_r,output_layer_activation,padding,start_neurons,train_loss,train_precision,train_accuracy,train_recall,val_loss,val_precision,val_accuracy,val_recall,train_f1,val_f1
0,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.500,17,5,0.0001,0.001,sigmoid,same,32,0.843425,0.928650,0.862717,0.857514,0.970667,0.868316,0.775568,0.778812,0.891666,0.821132
1,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.500,17,5,0.0001,0.001,sigmoid,same,32,0.856955,0.949212,0.845839,0.809346,0.980064,0.903571,0.766572,0.724409,0.873717,0.804132
2,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.400,17,5,0.0001,0.001,sigmoid,same,32,0.832348,0.944839,0.879387,0.867630,1.003364,0.888981,0.783144,0.768074,0.904590,0.824117
3,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.200,17,5,0.0001,0.001,sigmoid,same,32,0.759288,0.980216,0.920956,0.898200,1.083491,0.900166,0.794034,0.774517,0.937417,0.832628
4,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.100,17,5,0.0001,0.001,sigmoid,same,32,0.765186,0.966016,0.935662,0.935273,1.217005,0.874209,0.786458,0.790981,0.950396,0.830515
5,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.010,17,5,0.0001,0.001,sigmoid,same,32,0.782132,0.983592,0.944435,0.931207,1.338903,0.889803,0.787405,0.774517,0.956683,0.828167
6,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.001,17,5,0.0001,0.001,sigmoid,same,32,0.793265,0.982209,0.929980,0.910209,1.372538,0.886949,0.779356,0.763780,0.944839,0.820769
7,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.000,17,5,0.0001,0.001,sigmoid,same,32,1.044652,0.971432,0.922752,0.909552,1.513449,0.885833,0.776989,0.760916,0.939474,0.818637
8,64,33,0.01000,2,adam,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32,2.342889,0.822314,0.720338,0.734196,2.349560,0.814815,0.708807,0.724409,0.775760,0.766957
9,64,33,0.01000,2,rmsprop,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32,1.584430,0.763097,0.711021,0.813967,1.595288,0.757818,0.705492,0.815319,0.787711,0.785517


In [170]:
sorted_f1_df

,batch_size,epochs,learning_rate,num_layers,optimizer,activation,decay_sequence,dropout_rate,filters,kernel_size,l1_r,l2_r,output_layer_activation,padding,start_neurons,train_loss,train_precision,train_accuracy,train_recall,val_loss,val_precision,val_accuracy,val_recall,train_f1,val_f1
19,64,37,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32,0.710379,0.951974,0.920872,0.926614,0.963329,0.881151,0.815341,0.833214,0.939123,0.856512
10,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32,0.796258,0.968050,0.905414,0.885676,1.000532,0.909462,0.813920,0.798139,0.925033,0.850172
25,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0000,0.001,sigmoid,same,32,0.456250,0.971378,0.921750,0.908007,0.697848,0.890650,0.805398,0.804581,0.938624,0.845431
11,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,16,0.559328,0.946777,0.902908,0.903436,0.761700,0.874040,0.799716,0.814603,0.924599,0.843275
13,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,64,1.184011,0.948904,0.915775,0.921833,1.459759,0.866363,0.795455,0.816750,0.935173,0.840825
47,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,36,0.910195,0.951840,0.911742,0.912532,1.132424,0.878223,0.798295,0.802465,0.931772,0.838636
36,32,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32,0.719439,0.938639,0.903933,0.914316,0.929952,0.864355,0.794508,0.812976,0.926318,0.837878
51,64,33,0.01000,2,sgd,tanh,"[1, 4]",0.300,17,5,0.0001,0.001,sigmoid,same,32,0.862765,0.942918,0.909186,0.918035,1.087199,0.861698,0.792614,0.812976,0.930310,0.836628
14,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,128,1.946189,0.955922,0.913227,0.910284,2.231662,0.885600,0.794981,0.792412,0.932545,0.836419
29,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32,0.774593,0.961107,0.907754,0.896265,0.985839,0.893409,0.796402,0.785970,0.927554,0.836253


In [155]:
pd.to_numeric(df_f1['val_f1'], errors='coerce')

0     0.821132
1     0.804132
2     0.824117
3     0.832628
4     0.830515
5     0.828167
6     0.820769
7     0.818637
8     0.766957
9     0.785517
10    0.850172
11    0.843275
12    0.831845
13    0.840825
14    0.836419
15    0.809115
16    0.800293
17    0.790123
18    0.828972
19    0.856512
20    0.796238
21    0.796238
22    0.796238
23    0.796238
24    0.805288
25    0.845431
26    0.796238
27    0.796238
28    0.796238
29    0.836253
30   NaN      
31    0.761004
32    0.831517
33    0.784201
34    0.821750
35    0.806797
36    0.837878
37    0.833272
38    0.818396
39    0.789155
40    0.831131
41    0.815500
42    0.789035
43    0.583601
44    0.572993
45    0.829969
46    0.828561
47    0.838636
48    0.833555
49    0.835149
50    0.828306
51    0.836628
52    0.825959
53    0.832174
54    0.780612
55    0.775374
56    0.756599
57    0.712715
58    0.754318
59    0.806134
60    0.789657
61    0.795973
62    0.819888
63    0.761058
64   NaN      
Name: val_f1, dtype: floa

In [152]:
df_f1['val_f1'].isnull().values.any()

True

In [156]:
df_f1.columns

Index(['batch_size', 'epochs', 'learning_rate', 'num_layers', 'optimizer',
       'activation', 'decay_sequence', 'dropout_rate', 'filters',
       'kernel_size', 'l1_r', 'l2_r', 'output_layer_activation', 'padding',
       'start_neurons', 'train_loss', 'train_precision', 'train_accuracy',
       'train_recall', 'val_loss', 'val_precision', 'val_accuracy',
       'val_recall', 'train_f1', 'val_f1'],
      dtype='object')